In [ ]:
!pip3 install numpy imutils

In [1]:
import imutils
import os
import time
import cv2
import numpy as np
from imutils.video import FPS

In [2]:
YOLO_PATH="yolo-coco"
OUTPUT_FILE="output/outfile.avi"
labelsPath = os.path.join(YOLO_PATH, "coco.names")
LABELS = open(labelsPath).read().strip().split("\n")     


In [3]:
CONFIDENCE=0.5
THRESHOLD=0.3

np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")     

In [4]:
weightsPath = os.path.sep.join([YOLO_PATH, "yolov3.weights"])
configPath = os.path.sep.join([YOLO_PATH, "yolov3.cfg"])

In [5]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [8]:
vs = cv2.VideoCapture(0)
time.sleep(2.0)
fps = FPS().start()
writer = None
(W, H) = (None, None)

cnt=0

In [9]:
while True:
    cnt+=1
    (grabbed, frame) = vs.read()
    if not grabbed:
        break
    if W is None or H is None:
        (W,H) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > CONFIDENCE:
                box = detection[0:4]*np.array([W*2, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE,
        THRESHOLD)

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(OUTPUT_FILE, fourcc, 30,
            (frame.shape[1], frame.shape[0]), True)

        elap = (end - start)
        print("[INFO] single frame took {:.4f} seconds".format(elap))

    writer.write(frame)
    cv2.imshow("Frame", cv2.resize(frame, (800, 600)))
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

    fps.update()

fps.stop()


print("[INFO] elasqped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

cv2.destroyAllWindows()
print("[INFO] cleaning up...")
writer.release()
vs.release()
        

[INFO] single frame took 1.2761 seconds
[INFO] elasqped time: 54.27
[INFO] approx. FPS: 0.81
[INFO] cleaning up...
